In [23]:
%load_ext autoreload
%autoreload 2

import os
from uberduck_ml_dev.data.get import get_mels, get_pitches
from uberduck_ml_dev.data.data import RADTTS_DEFAULTS as data_config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
target_folder = '/usr/src/app/uberduck_ml_dev/analytics/tests/fixtures/ljtest/processed/'
source_folder = '/usr/src/app/uberduck_ml_dev/analytics/tests/fixtures/ljtest/wavs/'
paths = os.listdir(source_folder)

local_path_list = [os.path.join(source_folder, path) for path in paths]
folder_path_list = [os.path.join(target_folder, path.split('.wav')[0]) for path in paths]
resampled_normalized_path_list = [os.path.join(folder_path, 'resampled_normalized.pt') for folder_path in folder_path_list]
spectrogram_path_list = [os.path.join(target_folder, folder_path, 'spectrogram.pt') for folder_path in folder_path_list]

for folder_path in folder_path_list:
    os.makedirs(folder_path, exist_ok = True)

def load_resampled_floatnorm_audio(source_path):
    rate, data = read(source_path)
    if len(data.shape) > 1:
        rez_data = data[:,0] / np.abs(data[:,0]).max()
    else:
        rez_data = data / np.abs(data).max()
    output = librosa.resample(rez_data, orig_sr = rate, target_sr = 22050)
    return output

integer_normalize_audio = lambda x : np.asarray((x / np.abs(x).max()) * (MAX_WAV_VALUE - 1), dtype = np.int16)
def save_audio(data, filename, rate = 22050):
    write(filename, rate, data) # must be in this order
    
def resample_normalize(source_path, target_path):
    
    resample_floatnorm_mono_audio = load_resampled_floatnorm_audio(source_path)
    resample_intnorm_audio = integer_normalize_audio(resample_floatnorm_mono_audio)
    save_audio(resample_intnorm_audio, target_path)

In [20]:
from scipy.io.wavfile import read

from uberduck_ml_dev.data.get import get
import librosa
import numpy as np
from scipy.io.wavfile import write

MAX_WAV_VALUE = 32768
sr = 22050
get(resample_normalize,
    local_path_list,
    resampled_normalized_path_list,
    True)

In [21]:
get_mels(resampled_normalized_path_list, data_config, spectrogram_path_list)

In [31]:
get_pitches(resampled_normalized_path_list, data_config, folder_path_list, method = 'radtts')